# Copia esempio lab04 per verificare se ci fosse un'errore nel nostro training

In [1]:
import torch
from torch import nn
import cv2
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import random_split
from PIL import Image
import copy
import datetime

c:\Users\goate\miniconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Impossibile trovare la procedura specificata'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ConvertImageDtype(torch.float32),
    transforms.Resize((64, 64))
])

In [3]:
BATCH_SIZE = 32

def open_image(img):
    np_array = cv2.imread(filename=img, flags=cv2.IMREAD_GRAYSCALE)
    width, height = np_array.shape
    t = torch.from_numpy(np_array)
    t = torch.reshape(t, (1, width, height))
    return t

trainset = torchvision.datasets.ImageFolder(root='CVPR2023_project_2_and_3_data/train/', transform=transform, loader=open_image)

true_train_set, validation_train_set = random_split(trainset, (0.85, 0.15))

trainloader = torch.utils.data.DataLoader(true_train_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True) # batch size of 1 because we have to crop in order to get all images to same size (64x64), also see pin_memory optin
validloader = torch.utils.data.DataLoader(validation_train_set, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

testset = torchvision.datasets.ImageFolder(root='CVPR2023_project_2_and_3_data/test/', transform=transform, loader=open_image)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True) # batch size of 1 because we have to crop in order to get all images to same size (64x64), also see pin_memory optin

print(f'entire train folder: {len(trainset)}, entire test folder: {len(testset)}, splitted trainset: {len(true_train_set)},  splitted validset: {len(validation_train_set)}')

entire train folder: 1500, entire test folder: 2985, splitted trainset: 1275,  splitted validset: 225


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2, stride=2),
                torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2, stride=2),
                torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1),
                torch.nn.ReLU()
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(32*12*12, 15)
        )


    def forward(self, x):
        return self.classifier(self.conv(x))


model = CNN().to(device)
model

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=15, bias=True)
  )
)

In [6]:
def initialize_weigths_and_bias(m):
  if (type(m) == torch.nn.Linear) or (type(m) == torch.nn.Conv2d):
      torch.nn.init.constant_(m.bias.data, 0) # initialize weights
      torch.nn.init.normal_(m.weight.data, mean=0.0, std=0.01)
  

model.apply(initialize_weigths_and_bias)

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4608, out_features=15, bias=True)
  )
)

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train_one_epoch(epoch_index, loader):
  running_loss = 0

  for i,data in enumerate(loader):

    input, labels = data
    input = input.to(device)
    labels = labels.to(device)

    outputs = model(input)

    loss = loss_function(outputs, labels)
    running_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  return running_loss/(i+1)

EPOCHS = 30

print('Training loop...')
for epoch in range(EPOCHS):
  train_loss = train_one_epoch(epoch, trainloader)
  print(f'Epoch [{epoch + 1}/{EPOCHS}], Loss: {train_loss:.3f}')


Training loop...


c:\Users\goate\miniconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch [1/30], Loss: 2.709
Epoch [2/30], Loss: 2.709
Epoch [3/30], Loss: 2.708
Epoch [4/30], Loss: 2.708
Epoch [5/30], Loss: 2.708
Epoch [6/30], Loss: 2.708
Epoch [7/30], Loss: 2.708
Epoch [8/30], Loss: 2.709
Epoch [9/30], Loss: 2.708
Epoch [10/30], Loss: 2.709
Epoch [11/30], Loss: 2.708
Epoch [12/30], Loss: 2.708
Epoch [13/30], Loss: 2.708
Epoch [14/30], Loss: 2.708
Epoch [15/30], Loss: 2.708
Epoch [16/30], Loss: 2.708
Epoch [17/30], Loss: 2.709
Epoch [18/30], Loss: 2.709
Epoch [19/30], Loss: 2.708
Epoch [20/30], Loss: 2.708
Epoch [21/30], Loss: 2.708
Epoch [22/30], Loss: 2.708
Epoch [23/30], Loss: 2.708
Epoch [24/30], Loss: 2.708
Epoch [25/30], Loss: 2.708
Epoch [26/30], Loss: 2.708
Epoch [27/30], Loss: 2.708
Epoch [28/30], Loss: 2.708
Epoch [29/30], Loss: 2.708
Epoch [30/30], Loss: 2.708


In [11]:
# Evaluate the trained network on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in validloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test images: 4 %
